This notebook analyzes the performance improvements to model's 2 and 3. With
respect to particular key words or datasets.

The models are evaluated on the Kaggle competition's held out private validation
set.

Improvements on the larger Elsevier corpus will can be evaluated in the future,
but will require further validation as there isn't a ground truth.

### Model 3 

Model 3 is the fastest model of all the Kaggle Models. It is also the most 
rigid. It uses a set of heuristics to extract datasets from a new set of 
Documents. In looking for datasets in documents, it is actually quite limited
by employing a simple string search for the datasets.

An idea to improve this model, is to relax some of the contraints by using a
regular expression.

In [1]:
import re

import pandas as pd

import src.models.kaggle_model2 as km2
import src.models.kaggle_model3 as km3
import src.models.kaggle_model3_regex_inference as km3r
import src.evaluate.model as em
import src.data.kaggle_repository as kr

In [2]:
class MockRepo:
    def __init__(self, df):
        self.df = df
    def get_validation_data(self):
        return self.df

In [3]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [6]:
with open("../data/entity_classification/ncses_priorities.csv", "r") as f:
    ncses_priorities = [l.strip().split(",")[0].replace('"', '') for l in f.readlines()[1:]]
    ncses_priorities_cleaned = list(map(clean_text, ncses_priorities))

kaggle_validation = pd.read_csv("../data/kaggle/validation.csv")

In [7]:
kaggle_validation

,Id,PredictionString,Usage
0,7bfd8bb51,dbgap,Ignored
1,7d3e31302,sass|sass data|school and staffing sass data|s...,Private
2,3644f959a,foodaps|national household food acquisition an...,Public
3,ed3527cf3,database of genotypes and phenotypes|database ...,Ignored
4,236061129,dbgap,Ignored
...,...,...,...
8058,3cb8b0e09,database of genotypes and phenotypes|database ...,Ignored
8059,324fe1310,dbgap,Ignored
8060,39e66a274,1000 genomes project|1000 genomes project 1000...,Private
8061,c411b1b6c,dbgap|american cancer society cancer preventio...,Public


In [8]:
ncses_mask = kaggle_validation["PredictionString"].apply(lambda s: len(set(s.split("|")).intersection(set(ncses_priorities_cleaned)))>0)

ncses_mask.sum()

kaggle_validation_ncses = kaggle_validation.loc[ncses_mask, :]

In [9]:
valid_ids = kaggle_validation_ncses["Id"].values
kaggle_validation_ncses

,Id,PredictionString,Usage
46,d7d2d07d4,business r d and innovation survey|brdis|surve...,Public
57,b8f279039,business r d and innovation survey|r d|brdis|s...,Public
62,4b871ab7c,national survey of college graduates|decennial...,Public
64,152b44241,nscg|national survey of college graduates,Private
66,e18be232b,national survey of recent college graduates|na...,Public
...,...,...,...
8013,8799c68ad,nsf survey of federal funds for research and d...,Ignored
8019,c6fb73a43,national survey of college graduates|cgs gre s...,Public
8030,2355997d9,current population survey|nls72|nscg|national ...,Private
8040,ea59e8f5f,national survey of recent college graduates|be...,Public


In [10]:
repo = kr.KaggleRepository()
validation_dataframe = repo.get_validation_data()
validation_dataframe_ncses = validation_dataframe.loc[validation_dataframe["id"].isin(valid_ids), :]

In [31]:
config = {
    "model_path": "../models/kaggle_model3/baseline/params.txt",
}

In [33]:
evaluation = em.evaluate_model(
    MockRepo(validation_dataframe_ncses),
    km3.KaggleModel3(),
    config,
)
evaluation

/home/ryanhausen/repos/democratizingdata-ml-algorithms/src/models/kaggle_model3.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["model_prediction"] = df["text"].apply(infer_sample)
/home/ryanhausen/repos/democratizingdata-ml-algorithms/src/evaluate/model.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_dataframe["statistics"] = output.apply(calc_f, axis=1)



        Model Evaluation:

        - Run time: 6.743489503860474 seconds, avg: 0.017115455593554502 seconds per sample
        - True Postive Count: 554, avg: 1.4060913705583757 per sample
        - Precision: 0.6367816091954023
        - Recall: 0.17328745699092898
        

In [36]:
evaluation_regex = em.evaluate_model(
    MockRepo(validation_dataframe_ncses),
    km3r.Kaggle3RegexInference(config),
    config,
)
evaluation_regex

/home/ryanhausen/repos/democratizingdata-ml-algorithms/src/models/regex_model.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["model_prediction"] = df["text"].apply(infer_f)
/home/ryanhausen/repos/democratizingdata-ml-algorithms/src/evaluate/model.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_dataframe["statistics"] = output.apply(calc_f, axis=1)



        Model Evaluation:

        - Run time: 70.14516115188599 seconds, avg: 0.1780334039387969 seconds per sample
        - True Postive Count: 1966, avg: 4.98984771573604 per sample
        - Precision: 0.8920145190562614
        - Recall: 0.6184334696445423
        

### Model 2

Model 2 from the Kaggle competition can be broken into two parts. The first part
performs *entity extraction* and the second part performs *entity
classification*.

*Entity Extraction*

In the original submission, the entity extraction methodology was based on the
[Schwartz & Hearst (2003)](https://pubmed.ncbi.nlm.nih.gov/12603049/) algorithm
which is a algorithm for extracting abbreviations and definitions in biomedical
text.

*Entity Classification*

To classify the extracted entities, the Model 2 fined tuned a
[RobBERTa](https://arxiv.org/abs/1907.11692) model for sentence classification.
The training dataset was hand curated by the team and consists a mix of entities
that satisfy the entity extraction criteria. Some of which are datasets (e.g.
United State of America (USA) and National Educational Longitudinal Study
(NELS)).

**Improving Model 2**

To improve the approach in Model 2, we took a look at improving both the entity
extraction and the entity classification algorithms. At a high level, we can
imagine that the two parts of the algorithm have different purposes. Ideally
the entity extraction algorithm will produce a high recall and the entity
classification algorithm will produce a high precision.

*Entity Extraction*

The entity extraction algorithm was limited in two ways. First, the decision to
use the Schwartz & Hearst (2003) algorithm was a generally good choice, but will
neccsarily miss some entities. Namely, those that aren't idenitified using the
form LONG NAME (ABBREVIATION). This will by design limit recall. Additionally,
the implementation of the algorithm in the submission wasn't robust. To improve
recall, we drop the schwartz & hearst algorithm and instead use a regular
expression to extract entities. 

*Entity Classification*

To improve the entity classification algorithm, we also fine tune a RobBERTa
based transformer model, but also include the NCSES targeted datasets as
positive training samples.


In [ ]:
evaluation = em.evaluate_model(
    MockRepo(validation_dataframe_ncses),
    km2.KaggleModel2(),
    config,
)
evaluation